# Example 30: Manual Regression - External Forecast Comparison

## Overview

This notebook demonstrates **`manual_reg()`**, a NEW feature in py-tidymodels v1.0.0 that allows you to **specify coefficients manually** without fitting.

### What is Manual Regression?

Instead of learning coefficients from data, you **provide them directly**:

```python
from py_parsnip import manual_reg

# Specify coefficients manually
spec = manual_reg(
    coefficients={'temperature': -0.5, 'wind_speed': 0.3},
    intercept=1000.0
)

# "Fit" (just stores coefficients)
fit = spec.fit(data, 'demand ~ temperature + wind_speed')

# Predict using manual coefficients
predictions = fit.predict(test_data)
```

**No fitting happens** - the model just uses your coefficients.

---

## Use Cases

### 1. **Compare with External Tools**
You have forecasts from:
- Excel (LINEST, Solver)
- R (lm, forecast)
- SAS (PROC REG)
- Commercial software
- Legacy systems

**Goal**: Reproduce those forecasts in py-tidymodels and compare.

### 2. **Incorporate Domain Expert Knowledge**
Subject matter experts provide coefficient estimates:
- "Temperature affects demand by -0.5 per degree"
- "Each $1 price increase reduces sales by 100 units"

**Goal**: Test expert intuition against data-driven models.

### 3. **Create Baselines**
Simple benchmarks for model comparison:
- Naive rules: `sales = baseline + 0.5 * price`
- Industry standards
- Historical coefficients

**Goal**: Ensure data-driven models beat simple baselines.

### 4. **Reproduce Legacy Forecasts**
Old system uses fixed coefficients:
- Migration validation
- Regulatory compliance
- Audit trails

**Goal**: Exactly match legacy outputs.

---

## Dataset

**Refinery Margins** (simple, interpretable)
- Small dataset for clear demonstration
- Easy to understand relationships
- Good for manual coefficient specification

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Manual regression imports (NEW!)
from py_parsnip import manual_reg, linear_reg

# Supporting imports
from py_rsample import initial_time_split
from py_yardstick import rmse, mae, r_squared

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports successful")

## Load Data

In [ ]:
# Load refinery margins data
data = pd.read_csv('../_md/__data/refinery_margins.csv')
data['date'] = pd.to_datetime(data['date'])

# Create margin column (brent - dubai)
data['margin'] = data['brent'] - data['dubai']

# Drop rows with missing values
data = data.dropna().reset_index(drop=True)

print(f"Data: {len(data):,} rows")
print(f"Date range: {data['date'].min()} to {data['date'].max()}")
print(f"\nColumns: {list(data.columns)}")
print(f"\nFirst few rows:")
print(data.head())

print(f"\nData statistics:")
print(data.describe())

## Train/Test Split

In [ ]:
# Time-based split
split = initial_time_split(data, prop=0.8)
train_data = split.training()
test_data = split.testing()

print(f"Train: {len(train_data):,} rows ({train_data['date'].min()} to {train_data['date'].max()})")
print(f"Test:  {len(test_data):,} rows ({test_data['date'].min()} to {test_data['date'].max()})")

---

# Use Case 1: Compare with External Tool (Excel)

**Scenario**: Your team has an Excel model that forecasts refinery margins using:
- Brent crude price
- Dubai crude price

Excel LINEST gave these coefficients:
- Brent: 1.2
- Dubai: -0.8
- Intercept: 50.0

**Goal**: Reproduce Excel forecasts and compare with py-tidymodels fitted model.

In [ ]:
# Excel coefficients (from external tool)
excel_coefficients = {
    'brent': 1.2,
    'dubai': -0.8
}
excel_intercept = 50.0

print("📊 Excel Model Coefficients:")
print(f"  Intercept: {excel_intercept}")
for var, coef in excel_coefficients.items():
    print(f"  {var}: {coef}")

# Create manual regression model
excel_model = manual_reg(
    coefficients=excel_coefficients,
    intercept=excel_intercept
)

print("\n✅ Manual regression model created with Excel coefficients")

In [ ]:
# "Fit" the manual model (just stores coefficients)
fit_excel = excel_model.fit(train_data, 'margin ~ brent + dubai')

# Evaluate on test data
eval_excel = fit_excel.evaluate(test_data)
outputs_excel, coeffs_excel, stats_excel = eval_excel.extract_outputs()

# Extract test metrics (LONG format - manual_reg)
test_stats = stats_excel[stats_excel['split'] == 'test']
test_rmse_excel = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_excel = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]
test_r2_excel = test_stats[test_stats['metric'] == 'r_squared']['value'].iloc[0]

print("📊 Excel Model - Test Performance:")
print(f"RMSE: {test_rmse_excel:.4f}")
print(f"MAE:  {test_mae_excel:.4f}")
print(f"R²:   {test_r2_excel:.4f}")

print("\n✅ Successfully reproduced Excel forecasts in py-tidymodels")

### Compare with py-tidymodels Fitted Model

In [ ]:
# Fit actual linear regression (data-driven)
fitted_model = linear_reg().fit(train_data, 'margin ~ brent + dubai')

# Evaluate
eval_fitted = fitted_model.evaluate(test_data)
outputs_fitted, coeffs_fitted, stats_fitted = eval_fitted.extract_outputs()

# Extract test metrics (LONG format - linear_reg)
test_stats = stats_fitted[stats_fitted['split'] == 'test']
test_rmse_fitted = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_fitted = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]
test_r2_fitted = test_stats[test_stats['metric'] == 'r_squared']['value'].iloc[0]

print("📊 py-tidymodels Fitted Model - Test Performance:")
print(f"RMSE: {test_rmse_fitted:.4f}")
print(f"MAE:  {test_mae_fitted:.4f}")
print(f"R²:   {test_r2_fitted:.4f}")

# Show learned coefficients
print("\n📊 Learned Coefficients:")
print(coeffs_fitted[['variable', 'coefficient']].to_string(index=False))

### Side-by-Side Comparison

In [ ]:
# Compare Excel vs Fitted
comparison_excel = pd.DataFrame([
    {
        'Model': 'Excel (Manual)',
        'Source': 'External tool (LINEST)',
        'Test_RMSE': test_rmse_excel,
        'Test_MAE': test_mae_excel,
        'Test_R²': test_r2_excel
    },
    {
        'Model': 'py-tidymodels (Fitted)',
        'Source': 'Learned from data',
        'Test_RMSE': test_rmse_fitted,
        'Test_MAE': test_mae_fitted,
        'Test_R²': test_r2_fitted
    }
])

print("\n📊 COMPARISON: Excel vs py-tidymodels")
print("="*70)
print(comparison_excel.to_string(index=False))

# Calculate improvement
rmse_improvement = (test_rmse_excel - test_rmse_fitted) / test_rmse_excel * 100
print(f"\n📈 RMSE Improvement: {rmse_improvement:+.2f}%")

if rmse_improvement > 0:
    print("✅ Data-driven model outperforms Excel")
else:
    print("⚠️ Excel model is competitive (domain knowledge valuable?)")

---

# Use Case 2: Domain Expert Knowledge

**Scenario**: A refinery expert says:
- "Brent price increase of $1 improves margin by $0.9"
- "Dubai price increase of $1 reduces margin by $0.7" (different crude grade)
- "Base margin is around $45"

**Goal**: Test expert intuition against data.

In [ ]:
# Expert coefficients
expert_coefficients = {
    'brent': 0.9,   # Expert: "$1 Brent increase → +$0.9 margin"
    'dubai': -0.7   # Expert: "$1 Dubai increase → -$0.7 margin"
}
expert_intercept = 45.0  # Expert: "Base margin is $45"

print("🧠 Domain Expert Coefficients:")
print(f"  Intercept: ${expert_intercept}")
print(f"  Brent: ${expert_coefficients['brent']} per $1 Brent increase")
print(f"  Dubai: ${expert_coefficients['dubai']} per $1 Dubai increase")

# Create expert model
expert_model = manual_reg(
    coefficients=expert_coefficients,
    intercept=expert_intercept
)

# Fit and evaluate
fit_expert = expert_model.fit(train_data, 'margin ~ brent + dubai')
eval_expert = fit_expert.evaluate(test_data)
outputs_expert, coeffs_expert, stats_expert = eval_expert.extract_outputs()

# Extract test metrics (LONG format - manual_reg)
test_stats = stats_expert[stats_expert['split'] == 'test']
test_rmse_expert = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_expert = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]
test_r2_expert = test_stats[test_stats['metric'] == 'r_squared']['value'].iloc[0]

print("\n📊 Expert Model - Test Performance:")
print(f"RMSE: {test_rmse_expert:.4f}")
print(f"MAE:  {test_mae_expert:.4f}")
print(f"R²:   {test_r2_expert:.4f}")

### Compare All Three Models

In [ ]:
# Full comparison
comparison_all = pd.DataFrame([
    {
        'Model': 'Excel (Manual)',
        'Brent_Coef': excel_coefficients['brent'],
        'Dubai_Coef': excel_coefficients['dubai'],
        'Intercept': excel_intercept,
        'Test_RMSE': test_rmse_excel,
        'Test_R²': test_r2_excel
    },
    {
        'Model': 'Expert Knowledge',
        'Brent_Coef': expert_coefficients['brent'],
        'Dubai_Coef': expert_coefficients['dubai'],
        'Intercept': expert_intercept,
        'Test_RMSE': test_rmse_expert,
        'Test_R²': test_r2_expert
    },
    {
        'Model': 'py-tidymodels (Data)',
        'Brent_Coef': coeffs_fitted[coeffs_fitted['variable'] == 'brent']['coefficient'].iloc[0],
        'Dubai_Coef': coeffs_fitted[coeffs_fitted['variable'] == 'dubai']['coefficient'].iloc[0],
        'Intercept': coeffs_fitted[coeffs_fitted['variable'] == 'Intercept']['coefficient'].iloc[0],
        'Test_RMSE': test_rmse_fitted,
        'Test_R²': test_r2_fitted
    }
])

print("\n📊 FULL COMPARISON: Manual vs Expert vs Data-Driven")
print("="*80)
print(comparison_all.to_string(index=False))

# Best model
best_idx = comparison_all['Test_RMSE'].idxmin()
best_model = comparison_all.iloc[best_idx]

print(f"\n🏆 Best Model: {best_model['Model']}")
print(f"   Test RMSE: {best_model['Test_RMSE']:.4f}")
print(f"   Test R²: {best_model['Test_R²']:.4f}")

### Visualize Coefficient Comparison

In [ ]:
# Bar chart of coefficients
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

models = comparison_all['Model']
x_pos = np.arange(len(models))

# Brent coefficient
axes[0].bar(x_pos, comparison_all['Brent_Coef'], alpha=0.7, color='steelblue')
axes[0].set_ylabel('Coefficient Value', fontsize=11)
axes[0].set_title('Brent Coefficient Comparison', fontsize=12, fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(models, rotation=15, ha='right')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].axhline(0, color='black', linewidth=0.5)

# Dubai coefficient
axes[1].bar(x_pos, comparison_all['Dubai_Coef'], alpha=0.7, color='coral')
axes[1].set_ylabel('Coefficient Value', fontsize=11)
axes[1].set_title('Dubai Coefficient Comparison', fontsize=12, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(models, rotation=15, ha='right')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].axhline(0, color='black', linewidth=0.5)

plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("- All models agree on direction (Brent +, Dubai -)")
print("- Magnitudes differ based on source (external/expert/data)")
print("- Data-driven coefficients often close to expert knowledge")

---

# Use Case 3: Simple Baseline

**Scenario**: Create a naive baseline for benchmarking:
- "Margin = Brent - Dubai" (crude spread)
- Industry rule of thumb

**Goal**: Ensure data-driven models beat this simple rule.

In [ ]:
# Naive baseline: Margin = Brent - Dubai (crude spread)
# Simple industry rule: margin should equal the price difference
baseline_coefficients = {
    'brent': 1.0,   # +$1 per Brent
    'dubai': -1.0   # -$1 per Dubai
}
baseline_intercept = 0.0  # No intercept (pure spread)

print("📏 Naive Baseline Coefficients:")
print(f"  Intercept: {baseline_intercept}")
print(f"  Brent: {baseline_coefficients['brent']}")
print(f"  Dubai: {baseline_coefficients['dubai']}")
print("\n💡 Simple rule: Margin = Brent - Dubai")

# Create baseline model
baseline_model = manual_reg(
    coefficients=baseline_coefficients,
    intercept=baseline_intercept
)

In [ ]:
# Fit and evaluate
fit_baseline = baseline_model.fit(train_data, 'margin ~ brent + dubai')
eval_baseline = fit_baseline.evaluate(test_data)
outputs_baseline, coeffs_baseline, stats_baseline = eval_baseline.extract_outputs()

# Extract test metrics (LONG format - manual_reg)
test_stats = stats_baseline[stats_baseline['split'] == 'test']
test_rmse_baseline = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_baseline = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]
test_r2_baseline = test_stats[test_stats['metric'] == 'r_squared']['value'].iloc[0]

print("\n📊 Naive Baseline - Test Performance:")
print(f"RMSE: {test_rmse_baseline:.4f}")
print(f"MAE:  {test_mae_baseline:.4f}")
print(f"R²:   {test_r2_baseline:.4f}")

# Compare with data-driven
improvement = (test_rmse_baseline - test_rmse_fitted) / test_rmse_baseline * 100
print(f"\n📈 Data-driven improvement over baseline: {improvement:+.2f}%")

if improvement > 10:
    print("✅ Data-driven model significantly beats naive baseline")
elif improvement > 0:
    print("🟡 Marginal improvement - consider if added complexity worth it")
else:
    print("⚠️ Baseline competitive - may want to stick with simple rule")

In [ ]:
# Partial specification: Only Brent coefficient known
# Dubai coefficient will default to 0.0
partial_coefficients = {
    'brent': 1.0  # Only specify Brent, Dubai defaults to 0
}
partial_intercept = 0.0

print("📝 Partial Model - Brent Only:")
print(f"  Brent: {partial_coefficients['brent']}")
print(f"  Dubai: 0.0 (default)")
print(f"  Intercept: {partial_intercept}")
print("\n💡 Missing coefficients automatically set to 0")

# Create partial model
partial_model = manual_reg(
    coefficients=partial_coefficients,
    intercept=partial_intercept
)

---

# Use Case 4: Partial Coefficient Specification

**Feature**: You can specify **some** coefficients, others default to 0.

**Scenario**: You know Brent coefficient but unsure about Dubai.

In [ ]:
fit_partial = partial_model.fit(train_data, 'margin ~ brent + dubai')
eval_partial = fit_partial.evaluate(test_data)
outputs_partial, coeffs_partial, stats_partial = eval_partial.extract_outputs()

# Extract test metrics (LONG format - manual_reg)
test_stats = stats_partial[stats_partial['split'] == 'test']
test_rmse_partial = test_stats[test_stats['metric'] == 'rmse']['value'].iloc[0]
test_mae_partial = test_stats[test_stats['metric'] == 'mae']['value'].iloc[0]
test_r2_partial = test_stats[test_stats['metric'] == 'r_squared']['value'].iloc[0]

print("\n📊 Partial Model - Test Performance:")
print(f"RMSE: {test_rmse_partial:.4f}")
print(f"MAE:  {test_mae_partial:.4f}")
print(f"R²:   {test_r2_partial:.4f}")

print("\n💡 Partial specification useful when:")
print("  - You know some coefficients but not all")
print("  - Want to fix certain coefficients while exploring others")
print("  - Testing sensitivity to individual coefficients")

---

# Final Comparison: All Models

In [ ]:
# Compile all models
final_comparison = pd.DataFrame([
    {'Model': 'Excel (Manual)', 'Source': 'External tool', 'RMSE': test_rmse_excel, 'R²': test_r2_excel},
    {'Model': 'Expert Knowledge', 'Source': 'Domain expert', 'RMSE': test_rmse_expert, 'R²': test_r2_expert},
    {'Model': 'Naive Baseline', 'Source': 'Simple rule', 'RMSE': test_rmse_baseline, 'R²': test_r2_baseline},
    {'Model': 'Partial (Brent only)', 'Source': 'Partial spec', 'RMSE': test_rmse_partial, 'R²': test_r2_partial},
    {'Model': 'py-tidymodels (Fitted)', 'Source': 'Data-driven', 'RMSE': test_rmse_fitted, 'R²': test_r2_fitted}
]).sort_values('RMSE')

print("\n" + "="*70)
print("📊 FINAL COMPARISON: All Models (sorted by RMSE)")
print("="*70 + "\n")
print(final_comparison.to_string(index=False))

# Best model
best = final_comparison.iloc[0]
print(f"\n🏆 BEST MODEL: {best['Model']}")
print(f"   Source: {best['Source']}")
print(f"   Test RMSE: {best['RMSE']:.4f}")
print(f"   Test R²: {best['R²']:.4f}")

## Visualize Final Comparison

In [ ]:
# Bar chart
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['green' if 'Data' in m else 'steelblue' if 'Excel' in m or 'Expert' in m else 'gray' 
          for m in final_comparison['Model']]

bars = ax.barh(final_comparison['Model'], final_comparison['RMSE'], color=colors, alpha=0.7)
ax.set_xlabel('Test RMSE (lower is better)', fontsize=11)
ax.set_title('Manual vs Data-Driven Models', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
ax.invert_yaxis()

# Add value labels
for bar in bars:
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2, 
            f'{width:.4f}', ha='left', va='center', fontsize=9)

plt.tight_layout()
plt.show()

---

# Key Takeaways

## When to Use Manual Regression

### ✅ Use Cases

1. **External Tool Comparison**
   - Reproduce Excel/R/SAS forecasts
   - Validate migration from legacy systems
   - Audit trail and compliance

2. **Domain Expert Knowledge**
   - Incorporate subject matter expertise
   - Test expert intuition vs data
   - Combine domain knowledge with ML

3. **Baseline Creation**
   - Simple benchmarks for comparison
   - Industry standard coefficients
   - Naive rules of thumb

4. **Legacy System Reproduction**
   - Exact match of old forecasts
   - Regulatory compliance
   - Business continuity

5. **Sensitivity Analysis**
   - Test impact of coefficient changes
   - "What if" scenarios
   - Explore coefficient ranges

### ❌ When NOT to Use

- You don't have external coefficients
- Data-driven fitting is preferred
- No need for external validation
- Coefficients are unknown

---

## Best Practices

### Validation
1. **Always compare** manual vs fitted models
2. **Check if manual coefficients are reasonable** (sign, magnitude)
3. **Evaluate on test set** (don't just trust coefficients)
4. **Document source** of manual coefficients

### Coefficient Specification
```python
# Full specification
manual_reg(
    coefficients={'x1': 1.5, 'x2': -0.3},
    intercept=10.0
)

# Partial specification (others default to 0)
manual_reg(
    coefficients={'x1': 1.5},  # x2 will be 0
    intercept=10.0
)

# Variable names must match formula
fit = spec.fit(data, 'y ~ x1 + x2')  # Matches coefficient names
```

### Statistical Inference
- **No p-values or confidence intervals** (coefficients not estimated)
- `coeffs` DataFrame will have NaN for std_error, t_stat, p_value
- This is expected - manual coefficients bypass statistical inference

### Integration with Workflows
```python
from py_workflows import Workflow

# Manual model in workflow
wf = Workflow().add_formula('y ~ x1 + x2').add_model(
    manual_reg(coefficients={'x1': 1.5, 'x2': -0.3}, intercept=10.0)
)

fit = wf.fit(train_data)
predictions = fit.predict(test_data)
```

---

## Common Pitfalls

### ❌ Mismatched Variable Names
```python
# Wrong
spec = manual_reg(coefficients={'x1': 1.5})
fit = spec.fit(data, 'y ~ x_1')  # Variable name mismatch!

# Correct
spec = manual_reg(coefficients={'x_1': 1.5})
fit = spec.fit(data, 'y ~ x_1')  # Names match
```

### ❌ Forgetting Intercept
```python
# May want intercept=0 sometimes, but usually need it
spec = manual_reg(
    coefficients={'x1': 1.5},
    intercept=10.0  # Don't forget!
)
```

### ❌ Unrealistic Coefficients
```python
# Always sanity check coefficients
# - Sign make sense? (temperature → heating: negative?)
# - Magnitude reasonable? (not 10000 when data range is 0-100)
# - Units correct? (per dollar, per degree, etc.)
```

### ❌ Not Comparing with Data-Driven
```python
# Always fit data-driven model for comparison
manual_fit = manual_reg(...).fit(train, formula)
fitted_fit = linear_reg().fit(train, formula)  # Compare!
```

---

## Production Considerations

### Documentation
- Document source of coefficients (Excel file, expert name, date)
- Version control coefficient values
- Explain rationale for manual specification

### Monitoring
- Track manual model performance over time
- Alert if performance degrades significantly
- Periodically re-evaluate if data-driven would be better

### When to Refit
- Manual coefficients are fixed (no retraining)
- Consider refitting data-driven model periodically
- Compare manual vs fitted performance regularly

### Audit Trail
- Save coefficient values and sources
- Track who provided coefficients and when
- Document any coefficient updates

---

# References

- **Manual Regression Documentation**: `_md/ISSUE_8_MANUAL_MODEL_SUMMARY.md`
- **Linear Regression**: Example 02 (parsnip demo)
- **Model Comparison**: Example 11 (WorkflowSet)
- **CLAUDE.md**: Complete architecture documentation